In [1]:
import numpy as np
import time

In [2]:
# Griewank function definition
def griewank(x):
    part1 = np.sum(x ** 2) / 4000
    part2 = np.prod(np.cos(x / np.sqrt(np.arange(1, len(x) + 1))))
    return part1 - part2 + 1


In [3]:
# Parameters
D = 30  # Dimension
NP = 100  # Population size
F = 0.5  # Scaling factor
CR = 0.9  # Crossover probability
lower_bound = -600
upper_bound = 600
max_generations = 1500
tol = 1e-5

In [4]:
# Seed for reproducibility
np.random.seed(42)

In [5]:
# Initialize population
pop = np.random.uniform(low=lower_bound, high=upper_bound, size=(NP, D))
fitness = np.array([griewank(ind) for ind in pop])
best_idx = np.argmin(fitness)
best_vector = pop[best_idx].copy()
best_fitness = fitness[best_idx]
fitness_history = [best_fitness]


In [6]:
# DE Main Loop
generation = 0
start_time = time.time()
while generation < max_generations and best_fitness > tol:
    generation += 1

    for i in range(NP):
        # Mutation
        idxs = list(range(NP))
        idxs.remove(i)
        r1, r2, r3 = np.random.choice(idxs, size=3, replace=False)
        donor = pop[r1] + F * (pop[r2] - pop[r3])
        donor = np.clip(donor, lower_bound, upper_bound)

        # Crossover
        trial = np.empty(D)
        j_rand = np.random.randint(D)
        for j in range(D):
            if np.random.rand() < CR or j == j_rand:
                trial[j] = donor[j]
            else:
                trial[j] = pop[i][j]

        # Selection
        f_trial = griewank(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fitness:
                best_fitness = f_trial
                best_vector = trial.copy()
    fitness_history.append(best_fitness)

    if generation % 50 == 0 or best_fitness <= tol:
        print(f"Generation {generation:3d}: Best Fitness = {best_fitness:.6e}")

    if best_fitness <= tol:
        print(f"\nTerminated at generation {generation} (fitness ≤ {tol}).")
        break


end_time = time.time()
elapsed_time = end_time - start_time


Generation  50: Best Fitness = 8.433046e+01


Generation 100: Best Fitness = 2.427794e+01


Generation 150: Best Fitness = 5.297998e+00


Generation 200: Best Fitness = 1.734101e+00


Generation 250: Best Fitness = 1.207917e+00


...

Generation 550: Best Fitness = 3.333040e-03


Generation 600: Best Fitness = 7.239571e-04


Generation 650: Best Fitness = 1.463584e-04


Generation 700: Best Fitness = 2.563937e-05


Generation 741: Best Fitness = 9.381350e-06

Terminated at generation 741 (fitness ≤ 1e-05).


In [7]:
# ------------------ Final Output ------------------
print("\nFinal Result:")
print(f"Best Vector: {best_vector}")
print(f"Best Fitness: {best_fitness}")
print(f"Total Time Taken: {end_time - start_time:.4f} seconds")


Final Result:
Best Vector: [-0.00095271  0.00107778  0.00020939 -0.00251162 -0.00083414  0.00015486
  0.00121074  0.00064288 -0.00107302 -0.00321964 -0.00210013  0.00169588
  0.00720408  0.00176579  0.00108776  0.00747129 -0.00373916 -0.00196754
  0.00663836 -0.00193129  0.0012646   0.00155292 -0.0018135   0.0024865
 -0.00162349 -0.00039851 -0.00124887  0.00324757  0.0038755   0.00331423]
Best Fitness: 9.38134968453408e-06
Total Time Taken: 8.4276 seconds


In [8]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=fitness_history,
    x=list(range(len(fitness_history))),
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue'),
    marker=dict(size=4)
))

fig.update_layout(
    title='Differential Evolution on levy Function',
    xaxis_title='Generation',
    yaxis_title='Fitness Value',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True
)

fig.show()
